## Описание набора данных
Для выполнения лабораторной работы, как и в предыдущих лабораторных, используется набор данных **Automobile Dataset** (файл `Automobile.csv`, источник: Kaggle – Automobile Dataset).

Данные содержат информацию о характеристиках автомобилей: происхождении, мощности двигателя, массе, ускорении, объёме двигателя, расходе топлива и т.д.

## 1. Импорт библиотек и загрузка данных

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    classification_report,
    roc_auc_score,
    roc_curve
)

%matplotlib inline

data = pd.read_csv("Automobile.csv")
print(data.head())

## 2. Оценка пригодности датасета для классификации

### 2.1. Выбор целевого атрибута (метки)
В датасете есть категориальный атрибут **origin**, который отражает регион происхождения автомобиля.
Он принимает три значения: `usa`, `japan`, `europe`. Следовательно, `origin` подходит как **целевая метка класса**.

### 2.2. Проверка сбалансированности классов
Посчитаем количество объектов каждого класса и их долю.

In [ ]:
class_counts = data["origin"].value_counts()
class_share = data["origin"].value_counts(normalize=True) * 100

print("Количество по классам:\n", class_counts, "\n")
print("Доли классов (%):\n", class_share.round(2))

**Вывод по сбалансированности:**
Класс `usa` доминирует (примерно ~60%+), классы `japan` и `europe` представлены заметно меньше.
Это означает, что датасет **несбалансирован**, поэтому при оценке качества важно смотреть не только Accuracy,
но и **macro Precision/Recall/F1**, так как они учитывают качество по каждому классу равновесно.

## 3. Очистка и подготовка данных

### 3.1. Удаление дубликатов




In [ ]:
data = data.drop_duplicates()

### 3.2. Проверка пропусков

In [ ]:
print("Пропуски по столбцам:\n", data.isna().sum())

В наборе данных встречаются пропуски (чаще всего в `horsepower`).
Для корректного обучения моделей пропуски будут заполнены медианой (импутация).

## 4. Подготовка признаков и целевого столбца

В качестве признаков берем числовые характеристики.
Столбец name исключаем (это текстовый идентификатор модели и не является полезным числовым признаком).

In [ ]:
X = data.drop(columns=["origin", "name"])
y = data["origin"]

print("Форма X:", X.shape)
print("Классы y:", y.unique())

## 5. Разделение на обучающую и тестовую выборки
Данные делятся на train/test в пропорции 70/30.
Используем `stratify=y`, чтобы доли классов в train и test были одинаковыми.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.30,
    random_state=42,
    stratify=y
)

print("Train:", X_train.shape, "Test:", X_test.shape)

## 6. Обучение модели kNN

Для kNN важно масштабирование признаков (StandardScaler), иначе признаки с большим масштабом (например `weight`)
будут доминировать при вычислении расстояний.

Pipeline для kNN:
- заполнение пропусков медианой
- стандартизация признаков
- классификатор kNN

In [ ]:
knn_model = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ("clf", KNeighborsClassifier(n_neighbors=7, weights="distance"))
])

knn_model.fit(X_train, y_train)

## 7. Обучение модели дерева решений

Дерево решений не требует масштабирования.
Для уменьшения переобучения зададим ограничения: `max_depth` и `min_samples_leaf`.

Pipeline для дерева:
- заполнение пропусков медианой
- DecisionTreeClassifier

In [ ]:
tree_model = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("clf", DecisionTreeClassifier(
        random_state=42,
        max_depth=5,
        min_samples_leaf=5
    ))
])

tree_model.fit(X_train, y_train)

## 8. Оценка качества моделей

Будем считать:
- Accuracy
- Precision / Recall / F1 (macro и weighted)
- ROC-AUC (для многоклассовой классификации через One-vs-Rest)

In [ ]:
def evaluate(model, X_test, y_test):
    y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)

    p_macro, r_macro, f1_macro, _ = precision_recall_fscore_support(
        y_test, y_pred, average="macro", zero_division=0
    )
    p_w, r_w, f1_w, _ = precision_recall_fscore_support(
        y_test, y_pred, average="weighted", zero_division=0
    )

    # ROC-AUC для многоклассовой классификации (OvR)
    classes = model.classes_
    y_bin = label_binarize(y_test, classes=classes)
    y_proba = model.predict_proba(X_test)

    roc_auc = roc_auc_score(
        y_bin, y_proba,
        average="macro",
        multi_class="ovr"
    )

    return {
        "Accuracy": acc,
        "Precision_macro": p_macro,
        "Recall_macro": r_macro,
        "F1_macro": f1_macro,
        "F1_weighted": f1_w,
        "ROC_AUC_macro": roc_auc
    }, y_pred

knn_metrics, knn_pred = evaluate(knn_model, X_test, y_test)
tree_metrics, tree_pred = evaluate(tree_model, X_test, y_test)

knn_metrics, tree_metrics

### 8.1. Отчёт классификации (Precision/Recall/F1 по каждому классу)

In [ ]:
print("kNN classification report:\n")
print(classification_report(y_test, knn_pred))

print("\nDecision Tree classification report:\n")
print(classification_report(y_test, tree_pred))

### 8.2. Сравнительная таблица метрик

In [ ]:
comparison = pd.DataFrame([knn_metrics, tree_metrics], index=["kNN", "Decision Tree"])
comparison

## 9. ROC-кривые (One-vs-Rest) для kNN и дерева решений

Так как классов три, строим ROC отдельно для каждого класса (OvR):
- рассматриваем класс как "1", остальные как "0"
- строим ROC по вероятностям predict_proba

In [ ]:
def plot_multiclass_roc(model, X_test, y_test, title):
    classes = model.classes_
    y_bin = label_binarize(y_test, classes=classes)
    y_proba = model.predict_proba(X_test)

    plt.figure(figsize=(7,5))
    for i, cls in enumerate(classes):
        fpr, tpr, _ = roc_curve(y_bin[:, i], y_proba[:, i])
        plt.plot(fpr, tpr, label=f"{cls}")

    plt.plot([0, 1], [0, 1], linestyle="--")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title(title)
    plt.legend()
    plt.show()

plot_multiclass_roc(knn_model, X_test, y_test, "ROC-кривые (OvR) — kNN")
plot_multiclass_roc(tree_model, X_test, y_test, "ROC-кривые (OvR) — Decision Tree")

## 10. Объяснение результатов

1) **Дерево решений** часто показывает более высокие значения Accuracy и F1, потому что:
- умеет находить нелинейные правила (например, комбинации `weight`, `cylinders`, `displacement`),
- автоматически выделяет информативные пороги по признакам.

2) **kNN** зависит от расстояний в пространстве признаков:
- требует стандартизации,
- может ошибаться при перекрытии классов (когда автомобили разных регионов имеют похожие характеристики),
- при дисбалансе чаще “тянется” к доминирующему классу `usa`.

3) **Почему важно смотреть macro-F1:**
Из-за доминирования `usa` метрика Accuracy может выглядеть высокой даже тогда,
когда модель хуже распознаёт `japan` и `europe`.
Macro-метрики показывают среднее качество по классам без учета их размера.

4) **ROC-AUC:**
ROC-AUC (OvR) показывает, насколько хорошо модель ранжирует вероятности классов.
Даже при ошибках в жёстком выборе класса (argmax), ROC-AUC может быть высоким,
если модель в целом корректно “отличает” классы по вероятностям.

## Итоговый вывод о пригодности датасета

Датасет **пригоден для задачи классификации**, потому что:
- есть естественная целевая метка `origin` (3 класса),
- есть набор числовых признаков, потенциально связанных с происхождением автомобиля.

Однако датасет имеет особенности:
- классы **несбалансированы** (доминирует `usa`),
- есть пропуски (например, `horsepower`), требующие обработки.

После предобработки (импутации и масштабирования для kNN) датасет корректно используется для сравнения kNN и дерева решений.